# Making Pandas DataFrames from API Requests
In this example, we will use the U.S. Geological Survey's API to grab a JSON object of earthquake data and convert it to a `pandas.DataFrame`.

USGS API: https://earthquake.usgs.gov/fdsnws/event/1/

### Get Data from API

In [1]:
import datetime as dt
import pandas as pd
import requests

yesterday = dt.date.today() - dt.timedelta(days=1)
api = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
payload = {
    'format': 'geojson',
    'starttime': yesterday - dt.timedelta(days=30),
    'endtime': yesterday
}
response = requests.get(api, params=payload)

# let's make sure the request was OK
response.status_code

200

Response of 200 means OK, so we can pull the data out of the result. Since we asked the API for a JSON payload, we can extract it from the response with the `json()` method.

### Isolate the Data from the JSON Response
We need to check the structures of the response data to know where our data is.

In [2]:
earthquake_json = response.json()
earthquake_json.keys()

dict_keys(['type', 'metadata', 'features', 'bbox'])

The USGS API provides information about our request in the `metadata` key. Note that your result will be different, regardless of the date range you chose, because the API includes a timestamp for when the data was pulled:

In [3]:
earthquake_json['metadata']

{'generated': 1751350991000,
 'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2025-05-31&endtime=2025-06-30',
 'title': 'USGS Earthquakes',
 'status': 200,
 'api': '1.14.1',
 'count': 9876}

Each element in the JSON array `features` is a row of data for our dataframe.

In [4]:
type(earthquake_json['features'])

list

Your data will be different depending on the date you run this.

In [5]:
earthquake_json['features'][0]

{'type': 'Feature',
 'properties': {'mag': 4.2,
  'place': '62 km WNW of Abra Pampa, Argentina',
  'time': 1751241590842,
  'updated': 1751258578040,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/us7000q9mu',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=us7000q9mu&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': None,
  'alert': None,
  'status': 'reviewed',
  'tsunami': 0,
  'sig': 271,
  'net': 'us',
  'code': '7000q9mu',
  'ids': ',us7000q9mu,',
  'sources': ',us,',
  'types': ',origin,phase-data,',
  'nst': 18,
  'dmin': 1.773,
  'rms': 0.72,
  'gap': 89,
  'magType': 'mb',
  'type': 'earthquake',
  'title': 'M 4.2 - 62 km WNW of Abra Pampa, Argentina'},
 'geometry': {'type': 'Point', 'coordinates': [-66.2948, -22.6039, 241.472]},
 'id': 'us7000q9mu'}

### Convert to DataFrame
We need to grab the `properties` section out of every entry in the `features` JSON array to create our dataframe.

In [6]:
earthquake_properties_data = [
    quake['properties'] for quake in earthquake_json['features']
]
df = pd.DataFrame(earthquake_properties_data)
df.head()

,mag,place,time,updated,tz,url,detail,felt,cdi,mmi,...,ids,sources,types,nst,dmin,rms,gap,magType,type,title
0,4.20,"62 km WNW of Abra Pampa, Argentina",1751241590842,1751258578040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",us7000q9mu,",",us,",",origin,phase-data,",18.0,1.77300,0.72,89.0,mb,earthquake,"M 4.2 - 62 km WNW of Abra Pampa, Argentina"
1,0.53,"4 km WSW of Anza, CA",1751241340680,1751317467407,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ci41008999,",",ci,",",nearby-cities,origin,phase-data,scitech-link,",13.0,0.03111,0.09,97.0,ml,earthquake,"M 0.5 - 4 km WSW of Anza, CA"
2,3.80,"151 km S of Nikolski, Alaska",1751241150434,1751257904040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",us7000q9mt,",",us,",",origin,phase-data,",7.0,1.39800,0.39,244.0,mb,earthquake,"M 3.8 - 151 km S of Nikolski, Alaska"
3,4.60,"193 km WSW of Tual, Indonesia",1751240317942,1751256662040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",us7000q9mq,",",us,",",origin,phase-data,",29.0,3.69700,0.96,92.0,mb,earthquake,"M 4.6 - 193 km WSW of Tual, Indonesia"
4,4.50,Carlsberg Ridge,1751240095835,1751253556040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",us7000q9mr,",",us,",",origin,phase-data,",58.0,16.89800,0.77,68.0,mb,earthquake,M 4.5 - Carlsberg Ridge


### (Optional) Write Data to CSV

In [7]:
df.to_csv('earthquakes.csv', index=False)

<hr>
<div>
    <a href="./2-creating_dataframes.ipynb">
        <button style="float: left;">&#8592; Previous Notebook</button>
    </a>
    <a href="./4-inspecting_dataframes.ipynb">
        <button style="float: right;">Next Notebook &#8594;</button>
    </a>
</div>
<br>
<hr>